# k-meansによる分類
- [x] ファイル読み込み
- [x] 地点抽出
- [x] k-meansの使用
- [x] 全体、クラスごとにデータを作成
- [ ] 帰無仮説、対立仮説の作成
- [ ] 分散分析表の作成
- [ ]  F検定

## 目次

[ファイル読み込み](#file)
[抽出](#select)
[k-means](#kmean)
[データ作成](#split)
[仮説](#assume)
[分散分析表](#ANOVA)
[F検定](#kentei)  
  
  
[1要因の場合](#one-facter)

In [1]:
import pandas as pd
import openpyxl
import numpy as np
#from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import scipy.spatial.distance as dis
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as hierarchy
from scipy.stats import f

In [2]:
basepath ="/Users/kazuki/Documents/Study/data/ryukyu3/{}"
rflocate2=basepath.format("locations0416.csv")
rfuni=basepath.format("gram1/counter/{}.xlsx")

<a id="file"></a>  
# ファイル読み込み

In [3]:
uniAdf=pd.read_excel(rfuni.format("all"),header=0,index_col=0)

locatedf=pd.read_csv(rflocate2,header=0,index_col=0)
locates=list(locatedf.loc["24":"66","0"])
words=list(uniAdf.index)

uniAdf2=uniAdf.loc[:,locates]

<a id="kmean"></a>
# k-means

In [4]:
#分類数決定
n_c=3

In [5]:
clusterdata=uniAdf2.values
clusterdata=clusterdata.T
z=KMeans(n_clusters=n_c,random_state=1).fit_predict(clusterdata)
labeldata=z.tolist()

In [6]:
print(labeldata)
print(len(labeldata))

[2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
43


[次のステップ](#split)

# 各グループの中で最も他の地点と類似しているもの

1. ラベルデータごとに分類
2. その中で最も類似性が高いものを選ぶ(平均が高いもの)
3. データの要素内の平均の高いものかつ分散が低いものを選ぶ

In [ ]:
def cos_sim(v1, v2):
    v1a=np.array(v1,dtype="float64")
    v2a=np.array(v2,dtype="float64")
    return np.dot(v1a, v2a) / (np.linalg.norm(v1a) * np.linalg.norm(v2a))

In [ ]:
mostloc=[0 for i in range(4)]
gindex=[[j for j in range(len(locates)) if labeldata[j] == i] for i in range(4)]
glocs=[[locates[j] for j in range(len(locates)) if labeldata[j] == i] for i in range(4)]
gsims=[0 for i in range(4)]

In [ ]:
uniAdf3=uniAdf2.T

In [ ]:
for g in range(4):
    gdf=uniAdf3.iloc[gindex[g]]
    gsim=np.zeros((len(gdf),len(gdf)))
    for i in range(len(gindex[g])):
        for j in range(len(gindex[g])):
            gsim[i][j]=cos_sim(list(gdf.iloc[i,:]),list(gdf.iloc[j,:]))
#     print(gsim)
    gsims[g]=pd.DataFrame(gsim,index=glocs[g],columns=glocs[g])

In [ ]:
gsims[0]

In [ ]:
for g in range(4):
    gsims[g]["平均類似度"]=gsims[g].mean(axis=1)

In [ ]:
gsims[0]

In [ ]:
gsims_sorted=[0 for i in range(4)]
for g in range(4):
    mostvalue=gsims[g].sort_values("平均類似度",ascending=False)
    gsims_sorted[g]=mostvalue

In [ ]:
gsims_sorted[0]

In [ ]:
mostloc=[0 for i in range(4)]
for g in range(4):
    mostloc[g]=list(gsims_sorted[g])[0]

In [ ]:
print(mostloc)

In [ ]:
gdfs=[0 for i in range(4)]
for g in range(4):
    gdfs[g]=uniAdf2.iloc[:,gindex[g]]

### 3番目 単語ごとの平均、分散が高いものを選ぶ

In [ ]:
for g in range(4):
    cpgdfs=gdfs[g].iloc[:,0:3]
    avg=cpgdfs.mean(axis=1)
    variance=cpgdfs.var(axis=1)
    standarzation=cpgdfs.std(axis=1)
    gdfs[g]["平均"]=avg
    gdfs[g]["分散"]=variance
    gdfs[g]["標準偏差"]=standarzation
#     print(avg)

In [ ]:
#表示
gdfs[0][ gdfs[0]["平均"] != 0 ].sort_values(["標準偏差","平均"],ascending=[True,False])

gdfs[1][ gdfs[1]["平均"] != 0 ].sort_values(["標準偏差","平均"],ascending=[True,False])

gdfs[2][ gdfs[2]["平均"] != 0 ].sort_values(["標準偏差","平均"],ascending=[True,False])

In [ ]:
#分析表の作成
uniAdf4=uniAdf2
avg=uniAdf4.mean(axis=1)
variance=uniAdf4.var(axis=1)
standarzation=uniAdf4.std(axis=1)
uniAdf4["平均"]=avg
uniAdf4["分散"]=variance
uniAdf4["標準偏差"]=standarzation

In [ ]:
uniAdf4[ uniAdf4["平均"] != 0 ].sort_values(["平均"],ascending=[False])

<a id="split"></a>
# データ分類

In [7]:
uniAdf3=uniAdf2.T
gindex=[[j for j in range(len(locates)) if labeldata[j] == i] for i in range(n_c)]
glocs=[[locates[j] for j in range(len(locates)) if labeldata[j] == i] for i in range(n_c)]

gdfs=[0 for i in range(n_c)]
for g in range(n_c):
    gdfs[g]=uniAdf3.iloc[gindex[g],:]

<a id="assume"></a>
# 仮説

$
H_{g0}:帰無仮説:分類ごとの平均に差はない  
$  
$
H_{p0}:帰無仮説:発音ごとの平均に差はない
$

<a id="ANOVA"></a>
# 分散分析表

- [ ]全体変動および群間変動、郡内変動の計算
- [ ]自由度の計算
- [ ]平均平方和
- [ ]F値の計算
- [ ]分散分析表の作成

In [ ]:
#全変動
SST=uniAdf3.var(axis=0)*len(list(locates))#分散
#群間変動および郡内変動
SSGs=np.zeroz((n_c,len(words)))
SSEs=np.zeroz((n_c,len(words)))

for g in range(n_c):
    SSEs[g]=gdfs[g].var(axis=0)*len(glocs[g]) #郡内変動
    SSGs[g]=(gdfs[g].mean(axis=0)-uniAdf3.mean(axis=0))**2 * len(glocs[g]) #群間変動
    
#全変動=要因1+要因2+交互作用+残りのずれ

In [15]:
#全変動
SST=np.var(uniAdf3.values)*(n_c*len(words))
#要因1の平方和=水準Aの平方和+水準Bの平方和
AVT=np.mean(uniAdf3.values)
AV1=np.array([np.mean(gdfs[g].values) for g in range(n_c)])
SS1=sum([ (AV1[g]-AVT)**2* (len(glocs[g])*len(words)) for g in range(n_c) ])
# print("全平均{}".format(AVT))
# print("要因1の平均{}".format(AV1))
# print("要因1の変動{}".format(SS1))
#要因2の平方和
AV2=np.sum(np.array([gdfs[g].sum(axis=0) for g in range(n_c)]),axis=0)/len(locates)
SS2=sum([(AV2[w]-AVT)**2*len(locates) for w in range(len(words))] )

#交互作用
#各群のずれ
AVG=[gdfs[g].mean(axis=0) for g in range(n_c)]
SSG= np.sum(np.array([ [(AVG[g][w]-AVT)**2*len(glocs[g]) for w in range(len(words))] for g in range(n_c) ]))
# print(SS2)
# print(SSC)
SSC=SSG-SS1-SS2

#残差
VAE=[gdfs[g].var(axis=0) for g in range(n_c)]
# print(VAE)
SSE=np.sum(np.array([[VAE[g][w] for w in range(len(words)) ] for g in range(n_c)]))
print(SSE)
SSA=[SS1,SS2,SSC,SSE,SST]

427.61367127496146


In [16]:
#自由度
fx_t=len(locates)-1
fx_e=sum([len(glocs[g])-1 for g in range(n_c)]) #sum(glocs)-n_c
fx_1=n_c-1
fx_2=len(words)-1
fx_c=fx_1*fx_2
fxA=[fx_1,fx_2,fx_c,fx_e,fx_t]

In [17]:
#分散分析表の作成
ANOVAdf=pd.DataFrame([SSA,fxA], index=["平方和","自由度"], columns=["要因1","要因2","交互作用","残差","トータル"])

In [18]:
#分析表の表示
ANOVAdf=ANOVAdf.T

In [19]:
#平均平方の計算および分析表への記入
ANOVAdf["平均平方和"]=ANOVAdf["平方和"]/ANOVAdf["自由度"]

In [20]:
#表示
ANOVAdf

,平方和,自由度,平均平方和
要因1,1.901443,2.0,0.950721
要因2,151771.618172,85.0,1785.548449
交互作用,3499.145562,170.0,20.583209
残差,427.613671,40.0,10.690342
トータル,11054.484989,42.0,263.202024


Fの計算にはscipyを使用  
$
各F=\frac{要因の分散(各要因or交互作用)}{残差の分散}
$

In [30]:
#Fの計算
F1=ANOVAdf.at["要因1","平均平方和"]/ANOVAdf.at["残差","平均平方和"]
F2=ANOVAdf.at["要因2","平均平方和"]/ANOVAdf.at["残差","平均平方和"]
FC=ANOVAdf.at["交互作用","平均平方和"]/ANOVAdf.at["残差","平均平方和"]
#scipyのクラスを使用
FA=[F1,F2,FC,0,0]

In [31]:
ANOVAdf["F値"]=FA

In [32]:
ANOVAdf

,平方和,自由度,平均平方和,F値
要因1,1.901443,2.0,0.950721,0.088933
要因2,151771.618172,85.0,1785.548449,167.024449
交互作用,3499.145562,170.0,20.583209,1.925402
残差,427.613671,40.0,10.690342,0.000000
トータル,11054.484989,42.0,263.202024,0.000000


In [33]:
#F検定の比較
#有意確率の指定
# five = f.ppf(0.95, dfn, dfd) 採択確率、分子の自由度、分母の自由度
FP1=f.ppf(0.95, fx_e, fx_1)
FP2=f.ppf(0.95, fx_e, fx_2)
FPC=f.ppf(0.95, fx_e, fx_c)
print("要因1:{} 要因2:{} 交互作用:{}".format(FP1,FP2,FPC))

要因1:19.4707364323255 要因2:1.536146811214861 交互作用:1.4657486536794113


<a id="one-facter"></a>
# 1要因の場合

In [10]:
assumer=[]
for w in range(len(words)):
    #全変動
    SST=np.var(uniAdf3.values,axis=0)
    SST=SST[w]*len(locates)
    AVT=np.mean(uniAdf3.values,axis=0)
    AVT=AVT[w]
    # print(SST,AVT)
    #群間変動
    cong=[gdfs[g].iloc[:,w] for g in range(n_c)]
    AVG=[np.mean(cong[g]) for g in range(n_c)]
    SSG=[(AVG[g]-AVT)**2*len(glocs[g]) for g in range(n_c)]
    #郡内変動
    SSE=sum([np.var(cong[g].values) for g in range(n_c)])
    # print(AVG)
    #自由度
    fx_t=len(locates)-1
    fx_g=n_c-1
    fx_e=len(locates)-n_c
    #平均平方和
    ASSG=sum(SSG)/fx_g 
    ASSE=SSE/fx_e
    #F値の計算
    F=ASSG/ASSE
    #有意水準5%
    esF=f.ppf(0.95, fx_e, fx_g)
#     print(AVT)
    # print(AVG)
    # print(SSG)
    # print(sum(SSG))
    # print([len(glocs[g]) for g in range(n_c)])
    # print(SST,SSG,SSE)
    # print(fx_t,fx_g,fx_e)
    # print(ASSG,ASSE)
    if F > esF:
        print(words[w],F,esF)
        assumer.append(w)

-c 150.1077346106707 19.4707364323255
-v 339.6749026804945 19.4707364323255
? 36.70383958835563 19.4707364323255
?in2 46.88372093023255 19.4707364323255
?j 99.54991206580436 19.4707364323255
?m 132.5581395348837 19.4707364323255
?n2 133.06974111229692 19.4707364323255
a 310.33094040709875 19.4707364323255
a: 742.4810511182161 19.4707364323255
an 165.28507126781693 19.4707364323255
aq 19.534883720930225 19.4707364323255
d 129.04630302640953 19.4707364323255
dz 33.0622427996068 19.4707364323255
dz2 94.7714683844171 19.4707364323255
e 343.6919954101837 19.4707364323255
g 84.2435922578927 19.4707364323255
gw 27.386712449924552 19.4707364323255
h 140.30642101394787 19.4707364323255
hj 72.97232115229434 19.4707364323255
hw 36.13991555861414 19.4707364323255
i 170.6535571211596 19.4707364323255
i: 378.92645913248657 19.4707364323255
in 20.620301875631256 19.4707364323255
in2 77.11138310893512 19.4707364323255
k 80.97447323236841 19.4707364323255
kh 58.843609985065065 19.4707364323255
kw 367.5

/Users/kazuki/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars


In [11]:
#多重比較法 Tukeyの方法
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [12]:
A= np.array(gdfs[0].iloc[:,assumer[0]])
B= np.array(gdfs[1].iloc[:,assumer[0]])
C= np.array(gdfs[2].iloc[:,assumer[0]])
data_arr = np.hstack( (A,B,C) )

In [13]:
ind_arr=np.hstack(np.array([np.repeat(chr(ord("A")+g),len(glocs[g])) for g in range(n_c)]))

In [14]:
print(assumer)
print(len(assumer))

[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14, 15, 16, 20, 22, 23, 25, 26, 27, 31, 32, 33, 36, 37, 39, 40, 41, 42, 44, 45, 46, 47, 50, 51, 54, 55, 57, 58, 59, 60, 61, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81]
60


In [15]:
print(data_arr)
print(ind_arr)

[19 19 20 20 20 19 20 21 21 21 20 21 20 20 20 20 23 21 20 20 22 20 21 21
 20 20 21 25 20 27 21 25 23 24 25 26 26 26 25 28 26 19 19]
['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'B' 'B' 'B' 'B' 'B'
 'C' 'C' 'C' 'C' 'C' 'C' 'C']


In [16]:
print(pairwise_tukeyhsd(data_arr,ind_arr))

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  A      B     3.8581   1.4687 6.2474  True 
  A      C     3.4009   1.3262 5.4756  True 
  B      C    -0.4571  -3.3602 2.4459 False 
--------------------------------------------


In [17]:
anresults=pairwise_tukeyhsd(data_arr,ind_arr)
print(anresults.reject)

[ True  True False]


In [18]:
comb=[[],[],[]]
for w in assumer:
    A= np.array(gdfs[0].iloc[:,w])
    B= np.array(gdfs[1].iloc[:,w])
    C= np.array(gdfs[2].iloc[:,w])
    data_arr = np.hstack( (A,B,C) )
    ind_arr=np.hstack(np.array([np.repeat(chr(ord("A")+g),len(glocs[g])) for g in range(n_c)]))
    print("発音記号{}においての有意な差".format(words[w]))
    anresults=pairwise_tukeyhsd(data_arr,ind_arr)
    print(anresults.reject)
    for i,h in enumerate(anresults.reject):
        if h == True:
            comb[i].append(words[w])

発音記号-cにおいての有意な差
[ True  True False]
発音記号-vにおいての有意な差
[ True False  True]
発音記号?においての有意な差
[False False  True]
発音記号?in2においての有意な差
[False  True  True]
発音記号?jにおいての有意な差
[ True False False]
発音記号?mにおいての有意な差
[ True False  True]
発音記号?n2においての有意な差
[ True False  True]
発音記号aにおいての有意な差
[ True  True  True]
発音記号a:においての有意な差
[ True  True  True]
発音記号anにおいての有意な差
[ True  True  True]
発音記号aqにおいての有意な差
[False False False]
発音記号dにおいての有意な差
[False  True  True]
発音記号dzにおいての有意な差
[ True False False]
発音記号dz2においての有意な差
[False  True False]
発音記号eにおいての有意な差
[ True False  True]
発音記号gにおいての有意な差
[False  True False]
発音記号gwにおいての有意な差
[ True False False]
発音記号hにおいての有意な差
[ True  True False]
発音記号hjにおいての有意な差
[False False False]
発音記号hwにおいての有意な差
[False False  True]
発音記号iにおいての有意な差
[ True False  True]
発音記号i:においての有意な差
[ True  True  True]
発音記号inにおいての有意な差
[False False False]
発音記号in2においての有意な差
[False  True  True]
発音記号kにおいての有意な差
[ True False False]
発音記号khにおいての有意な差
[ True False  True]
発音記号kwにおいての有意な差
[ True  True False]
発音記号kwhにおいての有意な差
[False False F

In [17]:
print(comb)

[['-c', '-v', '?j', '?m', '?n2', 'a', 'a:', 'an', 'dz', 'e', 'gw', 'h', 'i', 'i:', 'k', 'kh', 'kw', 'kwx', 'kx', 'm', 'n', 'n2', 'o', 'o:', 'ph', 'pjh', 'pjx', 'px', 'q', 't', 'th', 'ts', 'ts2', 'ts2h', 'ts2x', 'tsh', 'tsx', 'tx', 'u', 'u:', 'ui', 'un', 'uq'], ['-c', '?in2', 'a', 'a:', 'an', 'd', 'dz2', 'g', 'h', 'i:', 'in2', 'kw', 'kwx', 'kx', 'm', 'n', 'n2', 'o', 'p', 'px', 'q', 'r', 't', 'th', 'ts2', 'ts2h', 'ts2x', 'tx', 'w'], ['-v', '?', '?in2', '?m', '?n2', 'a', 'a:', 'an', 'd', 'e', 'hw', 'i', 'i:', 'in2', 'kh', 'n', 'n2', 'o:', 'p', 'ph', 'pjx', 't', 'th', 'ts2x', 'u', 'u:', 'un', 'w']]


In [18]:
for com in comb:
    print(len(com))
    print(com)

43
['-c', '-v', '?j', '?m', '?n2', 'a', 'a:', 'an', 'dz', 'e', 'gw', 'h', 'i', 'i:', 'k', 'kh', 'kw', 'kwx', 'kx', 'm', 'n', 'n2', 'o', 'o:', 'ph', 'pjh', 'pjx', 'px', 'q', 't', 'th', 'ts', 'ts2', 'ts2h', 'ts2x', 'tsh', 'tsx', 'tx', 'u', 'u:', 'ui', 'un', 'uq']
29
['-c', '?in2', 'a', 'a:', 'an', 'd', 'dz2', 'g', 'h', 'i:', 'in2', 'kw', 'kwx', 'kx', 'm', 'n', 'n2', 'o', 'p', 'px', 'q', 'r', 't', 'th', 'ts2', 'ts2h', 'ts2x', 'tx', 'w']
28
['-v', '?', '?in2', '?m', '?n2', 'a', 'a:', 'an', 'd', 'e', 'hw', 'i', 'i:', 'in2', 'kh', 'n', 'n2', 'o:', 'p', 'ph', 'pjx', 't', 'th', 'ts2x', 'u', 'u:', 'un', 'w']


In [26]:
print(set(comb[0]))

{'n2', 'h', 'u', 'u:', 'th', 'ph', 'a', 'kh', 'kx', 'q', '-v', 'e', '?n2', '?m', 'm', 'a:', 'o', 'un', 'o:', 'pjx', 'pjh', '-c', 'ts2x', 'tsx', 'ui', 'ts2h', 'kw', 'n', 'px', 'ts', 'dz', '?j', 'an', 't', 'gw', 'tsh', 'i', 'uq', 'i:', 'ts2', 'tx', 'kwx', 'k'}


In [27]:
ABw=set(comb[0])
ACw=set(comb[1])
BCw=set(comb[2])
print("Aの特徴ともいえる要素数{}と要素{}".format(len(ABw | ACw),ABw | ACw))
print("Bの特徴ともいえる要素数{}と要素{}".format(len(ABw | BCw),ABw | BCw))
print("Cの特徴ともいえる要素数{}と要素{}".format(len(ACw | BCw),ACw | BCw))

Aの特徴ともいえる要素数51と要素{'n2', 'h', 'in2', 'u', 'u:', 'th', 'ph', 'a', 'kh', 'kx', 'q', '-v', 'e', '?n2', '?m', 'm', 'd', 'a:', 'g', 'o', 'un', 'o:', 'pjx', 'pjh', '-c', 'ts2x', 'tsx', 'dz2', 'ui', 'r', 'ts2h', 'kw', 'n', 'p', 'px', 'ts', 'dz', '?in2', '?j', 'an', 't', 'w', 'gw', 'tsh', 'i', 'uq', 'i:', 'ts2', 'tx', 'kwx', 'k'}
Bの特徴ともいえる要素数50と要素{'n2', 'h', 'in2', 'u', 'u:', 'th', 'ph', 'a', 'hw', 'kh', 'kx', '?', 'q', '-v', 'e', '?n2', '?m', 'm', 'd', 'a:', 'o', 'un', 'o:', 'pjx', 'pjh', '-c', 'ts2x', 'tsx', 'ui', 'ts2h', 'kw', 'n', 'p', 'px', 'ts', 'dz', '?in2', '?j', 'an', 't', 'w', 'gw', 'tsh', 'i', 'uq', 'i:', 'ts2', 'tx', 'kwx', 'k'}
Cの特徴ともいえる要素数43と要素{'n2', 'h', 'in2', 'u', 'u:', 'th', 'a', 'hw', 'kx', 'kh', '?', 'q', '-v', 'e', '?n2', 'm', '?m', 'd', 'a:', 'g', 'o', 'un', 'o:', 'ts2x', 'pjx', '-c', 'dz2', 'r', 'p', 'kw', 'n', 'ts2h', 'px', '?in2', 'an', 't', 'w', 'i', 'i:', 'ts2', 'tx', 'kwx', 'ph'}


In [28]:
print(len(words))

86


In [29]:
print(len(comb[0]))
print(len(comb[1]))
print(len(comb[2]))

43
29
28


In [39]:
resultsdf=pd.DataFrame(sorted(list(ABw | ACw)))

In [40]:
resultgroup2=np.array(sorted(list(ABw | BCw)))
resultgroup2=np.append(resultgroup2,np.repeat(" ",len(ABw | ACw)-len(ABw | BCw)))
# np.repeat(" ",len(ABw | ACw)-len(ABw | BCw))
resultgroup3=np.array(sorted(list(ACw | BCw)))
resultgroup3=np.append(resultgroup3,np.repeat(" ",len(ABw | ACw)-len(ACw | BCw)))
# np.repeat(" ",len(ABw | ACw)-len(ACw | BCw))

In [41]:
resultsdf["1"]=resultgroup2
resultsdf["2"]=resultgroup3

In [42]:
print(resultsdf.to_latex())

\begin{tabular}{llll}
\toprule
{} &     0 &     1 &     2 \\
\midrule
0  &    -c &    -c &    -c \\
1  &    -v &    -v &    -v \\
2  &  ?in2 &     ? &     ? \\
3  &    ?j &  ?in2 &  ?in2 \\
4  &    ?m &    ?j &    ?m \\
5  &   ?n2 &    ?m &   ?n2 \\
6  &     a &   ?n2 &     a \\
7  &    a: &     a &    a: \\
8  &    an &    a: &    an \\
9  &     d &    an &     d \\
10 &    dz &     d &   dz2 \\
11 &   dz2 &    dz &     e \\
12 &     e &     e &     g \\
13 &     g &    gw &     h \\
14 &    gw &     h &    hw \\
15 &     h &    hw &     i \\
16 &     i &     i &    i: \\
17 &    i: &    i: &   in2 \\
18 &   in2 &   in2 &    kh \\
19 &     k &     k &    kw \\
20 &    kh &    kh &   kwx \\
21 &    kw &    kw &    kx \\
22 &   kwx &   kwx &     m \\
23 &    kx &    kx &     n \\
24 &     m &     m &    n2 \\
25 &     n &     n &     o \\
26 &    n2 &    n2 &    o: \\
27 &     o &     o &     p \\
28 &    o: &    o: &    ph \\
29 &     p &     p &   pjx \\
30 &    ph &    ph &    px \\


In [44]:
resultLocate1=np.array(glocs[0])
resultLocate2=np.array(glocs[1])
resultLocate2=np.append(resultLocate2,np.repeat(" ",len(glocs[0])-len(glocs[1])))
resultLocate3=np.array(glocs[2])
resultLocate3=np.append(resultLocate3,np.repeat(" ",len(glocs[0])-len(glocs[2])))
resultLocates=np.vstack((resultLocate1,resultLocate2,resultLocate3))
resultLocateDf=pd.DataFrame(resultLocates)

In [46]:
print(resultLocateDf.T.to_latex())

\begin{tabular}{llll}
\toprule
{} &      0 &      1 &      2 \\
\midrule
0  &   読谷古堅 &  今帰与那嶺 &  国頭辺土名 \\
1  &   石川石川 &   名護川上 &  国頭辺野喜 \\
2  &   沖縄胡屋 &   名護大兼 &  大宜根路銘 \\
3  &  うる具志川 &   恩納恩納 &   本部備瀬 \\
4  &  うる与那城 &   金武金武 &   本部浜元 \\
5  &  うる平安座 &        &   うる津堅 \\
6  &   北谷北谷 &        &   南城久高 \\
7  &   北中屋宜 &        &        \\
8  &  宜湾普天間 &        &        \\
9  &   宜湾大山 &        &        \\
10 &  浦添安波茶 &        &        \\
11 &  西原小波津 &        &        \\
12 &  那覇当蔵士 &        &        \\
13 &  那覇当蔵平 &        &        \\
14 &  那覇泉崎士 &        &        \\
15 &  那覇泉崎平 &        &        \\
16 &   那覇安里 &        &        \\
17 &   南風兼城 &        &        \\
18 &  与那与那原 &        &        \\
19 &  南城古堅2 &        &        \\
20 &   南城佐敷 &        &        \\
21 &   南城知念 &        &        \\
22 &   南城玉城 &        &        \\
23 &   南城糸数 &        &        \\
24 &  八重具志頭 &        &        \\
25 &   八重友寄 &        &        \\
26 &   糸満米須 &        &        \\
27 &  糸満喜屋武 &        &        \\
28 

In [19]:
wls=np.full((3,len(words))," ")
sortdataDf=pd.DataFrame(wls,columns=words)
sortdataDf

,-c,-v,?,?in2,?j,?m,?n2,?n2:,a,a:,...,u:,ui,un,uq,v,w,y,y:,yq,z
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,


In [20]:
sortdataDf.loc[0,comb[0]]="True"
sortdataDf.loc[1,comb[1]]="True"
sortdataDf.loc[2,comb[2]]="True"

In [21]:
sortdataDf

,-c,-v,?,?in2,?j,?m,?n2,?n2:,a,a:,...,u:,ui,un,uq,v,w,y,y:,yq,z
0,True,True,,,True,True,True,,True,True,...,True,True,True,True,,,,,,
1,True,,,True,,,,,True,True,...,,,,,,True,,,,
2,,True,True,True,,True,True,,True,True,...,True,,True,,,True,,,,


In [23]:
# sortdataDf1=(sortdataDf==1)
print(sortdataDf.T.to_latex())

\begin{tabular}{llll}
\toprule
{} &     0 &     1 &     2 \\
\midrule
-c   &  True &  True &       \\
-v   &  True &       &  True \\
?    &       &       &  True \\
?in2 &       &  True &  True \\
?j   &  True &       &       \\
?m   &  True &       &  True \\
?n2  &  True &       &  True \\
?n2: &       &       &       \\
a    &  True &  True &  True \\
a:   &  True &  True &  True \\
an   &  True &  True &  True \\
aq   &       &       &       \\
b    &       &       &       \\
d    &       &  True &  True \\
dz   &  True &       &       \\
dz2  &       &  True &       \\
e    &  True &       &  True \\
e2   &       &       &       \\
e:   &       &       &       \\
f    &       &       &       \\
g    &       &  True &       \\
gj   &       &       &       \\
gw   &  True &       &       \\
h    &  True &  True &       \\
h!   &       &       &       \\
hj   &       &       &       \\
hw   &       &       &  True \\
i    &  True &       &  True \\
i2   &       &       &       \\
i2

In [42]:
print(len(list(set(comb[0])|set(comb[1])|set(comb[2]))))

53


In [49]:
print( sorted(list(set(comb[0])|set(comb[1])|set(comb[2]))) )

['-c', '-v', '?', '?in2', '?j', '?m', '?n2', 'a', 'a:', 'an', 'd', 'dz', 'dz2', 'e', 'g', 'gw', 'h', 'hw', 'i', 'i:', 'in2', 'k', 'kh', 'kw', 'kwx', 'kx', 'm', 'n', 'n2', 'o', 'o:', 'p', 'ph', 'pjh', 'pjx', 'px', 'q', 'r', 't', 'th', 'ts', 'ts2', 'ts2h', 'ts2x', 'tsh', 'tsx', 'tx', 'u', 'u:', 'ui', 'un', 'uq', 'w']


In [50]:
nanika=[words[amr] for amr in assumer]
print(nanika)

['-c', '-v', '?', '?in2', '?j', '?m', '?n2', 'a', 'a:', 'an', 'aq', 'd', 'dz', 'dz2', 'e', 'g', 'gw', 'h', 'hj', 'hw', 'i', 'i:', 'in', 'in2', 'k', 'kh', 'kw', 'kwh', 'kwx', 'kx', 'm', 'n', 'n2', 'n2:', 'o', 'o:', 'p', 'ph', 'pjh', 'pjx', 'px', 'q', 'r', 's2', 't', 'th', 'ts', 'ts2', 'ts2h', 'ts2x', 'ts3', 'tsh', 'tsx', 'tx', 'u', 'u:', 'ui', 'un', 'uq', 'w']


In [62]:
tyuuDf=uniAdf2.loc[nanika,:]
clusterdata=tyuuDf.values
clusterdata=clusterdata.T
z=KMeans(n_clusters=n_c,random_state=1).fit_predict(clusterdata)
labeldata=z.tolist()
print(labeldata)
print(len(labeldata))

[2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
43


In [63]:
kresultLocate1=np.array(glocs[0])
kresultLocate2=np.array(glocs[1])
kresultLocate2=np.append(kresultLocate2,np.repeat(" ",len(glocs[0])-len(glocs[1])))
kresultLocate3=np.array(glocs[2])
kresultLocate3=np.append(kresultLocate3,np.repeat(" ",len(glocs[0])-len(glocs[2])))
kresultLocates=np.vstack((kresultLocate1,kresultLocate2,kresultLocate3))
kresultLocateDf=pd.DataFrame(kresultLocates)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,読谷古堅,石川石川,沖縄胡屋,うる具志川,うる与那城,うる平安座,北谷北谷,北中屋宜,宜湾普天間,宜湾大山,...,南城知念,南城玉城,南城糸数,八重具志頭,八重友寄,糸満米須,糸満喜屋武,糸満真壁,糸満国吉,糸満兼城
1,今帰与那嶺,名護川上,名護大兼,恩納恩納,金武金武,,,,,,...,,,,,,,,,,
2,国頭辺土名,国頭辺野喜,大宜根路銘,本部備瀬,本部浜元,うる津堅,南城久高,,,,...,,,,,,,,,,


In [65]:
print(kresultLocateDf.T.to_latex())

\begin{tabular}{llll}
\toprule
{} &      0 &      1 &      2 \\
\midrule
0  &   読谷古堅 &  今帰与那嶺 &  国頭辺土名 \\
1  &   石川石川 &   名護川上 &  国頭辺野喜 \\
2  &   沖縄胡屋 &   名護大兼 &  大宜根路銘 \\
3  &  うる具志川 &   恩納恩納 &   本部備瀬 \\
4  &  うる与那城 &   金武金武 &   本部浜元 \\
5  &  うる平安座 &        &   うる津堅 \\
6  &   北谷北谷 &        &   南城久高 \\
7  &   北中屋宜 &        &        \\
8  &  宜湾普天間 &        &        \\
9  &   宜湾大山 &        &        \\
10 &  浦添安波茶 &        &        \\
11 &  西原小波津 &        &        \\
12 &  那覇当蔵士 &        &        \\
13 &  那覇当蔵平 &        &        \\
14 &  那覇泉崎士 &        &        \\
15 &  那覇泉崎平 &        &        \\
16 &   那覇安里 &        &        \\
17 &   南風兼城 &        &        \\
18 &  与那与那原 &        &        \\
19 &  南城古堅2 &        &        \\
20 &   南城佐敷 &        &        \\
21 &   南城知念 &        &        \\
22 &   南城玉城 &        &        \\
23 &   南城糸数 &        &        \\
24 &  八重具志頭 &        &        \\
25 &   八重友寄 &        &        \\
26 &   糸満米須 &        &        \\
27 &  糸満喜屋武 &        &        \\
28 

In [73]:
nowords=list(set(words)-set(nanika))
print(nowords)

['j', 'y:', 'l', 's', 'iq', 'nj', 'tjx', 'on', 'kj', 'z', 'yq', 'i2n', 'i2', 'pj', 'gj', 'b', 'y', 'e:', 'e2', 'oq', 'v', 'ng', 'f', 'h!', 'i2:', '?n2:']


In [95]:
for w in nowords:
    #全変動
    aa=words.index(w)
    AVT=np.mean(uniAdf3.values,axis=0)
    AVT=AVT[aa]
    #郡内変動
    cong=[gdfs[g].iloc[:,aa] for g in range(n_c)]
    AVG=[np.mean(cong[g]) for g in range(n_c)]
#     SSE=sum([np.var(cong[g].values) for g in range(n_c)])
    print("{} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\ ".format(w,AVT,AVG[0],AVG[1],AVG[2]))

j & 2.12 & 2.13 & 2.00 & 2.14 \\ 
y: & 0.00 & 0.00 & 0.00 & 0.00 \\ 
l & 0.00 & 0.00 & 0.00 & 0.00 \\ 
s & 2.77 & 2.77 & 2.80 & 2.71 \\ 
iq & 0.05 & 0.03 & 0.00 & 0.14 \\ 
nj & 0.00 & 0.00 & 0.00 & 0.00 \\ 
tjx & 0.00 & 0.00 & 0.00 & 0.00 \\ 
on & 0.00 & 0.00 & 0.00 & 0.00 \\ 
kj & 0.00 & 0.00 & 0.00 & 0.00 \\ 
z & 0.00 & 0.00 & 0.00 & 0.00 \\ 
yq & 0.00 & 0.00 & 0.00 & 0.00 \\ 
i2n & 0.00 & 0.00 & 0.00 & 0.00 \\ 
i2 & 0.02 & 0.03 & 0.00 & 0.00 \\ 
pj & 0.00 & 0.00 & 0.00 & 0.00 \\ 
gj & 0.00 & 0.00 & 0.00 & 0.00 \\ 
b & 1.88 & 1.90 & 1.60 & 2.00 \\ 
y & 0.00 & 0.00 & 0.00 & 0.00 \\ 
e: & 1.07 & 1.00 & 1.20 & 1.29 \\ 
e2 & 0.00 & 0.00 & 0.00 & 0.00 \\ 
oq & 0.00 & 0.00 & 0.00 & 0.00 \\ 
v & 0.00 & 0.00 & 0.00 & 0.00 \\ 
ng & 0.00 & 0.00 & 0.00 & 0.00 \\ 
f & 0.00 & 0.00 & 0.00 & 0.00 \\ 
h! & 0.00 & 0.00 & 0.00 & 0.00 \\ 
i2: & 0.00 & 0.00 & 0.00 & 0.00 \\ 
?n2: & 0.07 & 0.03 & 0.00 & 0.29 \\ 
